In [1]:
import numpy as np
import json
import time
from DPKL import DPKL
from utils import data_splitter
from computations import get_covg
from scipy.io import loadmat

No GPU found


In [2]:
#Model Hyperparameters (change as necessary)
with open('params.json', 'r') as f:
    hp_dict = json.load(f)

In [3]:
hp_dict

{'GP_reg': 1.0,
 'SS_reg': 1.0,
 'latent_dim': 2,
 'lr': 0.001,
 'num_iters': 50,
 'num_samples': 10}

In [4]:
#Experiment Hyperparamters (change as necessary)
datapath = '../../../../ssdkl_uci_data/'
datasets = ['ctslice']
num_trials = 1
num_test = 1000
num_lab_list = [50, 100, 200, 300, 400, 500]
ss_flag = False #True for semi-supervised

In [5]:
#Fixing RNG seed for fair comparison
np.random.seed(0)

In [6]:
rmse_mean = np.zeros((len(num_lab_list),len(datasets)))
rmse_std = np.zeros((len(num_lab_list),len(datasets)))
negll_mean = np.zeros((len(num_lab_list),len(datasets)))
negll_std = np.zeros((len(num_lab_list),len(datasets)))

In [7]:
start_time = time.time()
for (j,dataset) in enumerate(datasets):
    filepath = datapath+dataset+'/'
    X_all = np.load(filepath+'X.npy').astype('float32')
    y_all = np.load(filepath+'y.npy').astype('float32')
    y_all = np.reshape(y_all,(-1,1))
    X_test, y_test, X_train, y_train = data_splitter(X_all, y_all, num_test) #Splitting into training and test data
    print (dataset)
    for (i,num_lab) in enumerate(num_lab_list):
        rmse_vect = np.zeros(num_trials)
        negll_vect = np.zeros(num_trials)
        for trial in range(num_trials):
            print ('Trial = '+str(trial))
            X_lab, y_lab, X_unlab, y_unlab = data_splitter(X_train, y_train, num_lab) #Splitting into labeled and unlabeled data
            DPKL_model = DPKL(hp_dict)
            if ss_flag:
                X_unlab = X_unlab[:min(X_unlab.shape[0],10000)]
                y_unlab = y_unlab[:min(X_unlab.shape[0],10000)]
                DPKL_model.fit(X_lab,y_lab,X_unlab)
            else:
                DPKL_model.fit(X_lab,y_lab)
            test_mean, test_Kmat = DPKL_model.predict(X_test)
            rmse_vect[trial] = np.sqrt(np.mean(np.square(y_test - test_mean)))
            y_test_rs = (y_test - test_mean)/DPKL_model.norm_params[1]
            test_var = np.diag(test_Kmat)
            negll_vect[trial] = get_covg(y_test_rs,test_var)
        rmse_mean[i][j] = np.mean(rmse_vect)
        rmse_std[i][j] = np.std(rmse_vect)
        negll_mean[i][j] = np.mean(negll_vect)
        negll_std[i][j] = np.std(negll_vect)
        print ('n = '+str(num_lab)+' RMSE mean = '+str(rmse_mean[i][j])+' RMSE stdev = '+str(rmse_std[i][j]))
        print ('n = '+str(num_lab)+' Negll mean = '+str(negll_mean[i][j])+' Negll stdev = '+str(negll_std[i][j]))
stop_time = time.time()

time_hrs = (stop_time - start_time)/3600
print ('Time taken = '+str(time_hrs)+' hours')

ctslice
Trial = 0
n = 50 RMSE mean = 14.082666397094727 RMSE stdev = 0.0
n = 50 Negll mean = 1.1594152450561523 Negll stdev = 0.0
Trial = 0
n = 100 RMSE mean = 10.222580909729004 RMSE stdev = 0.0
n = 100 Negll mean = 1.0411406755447388 Negll stdev = 0.0
Trial = 0
n = 200 RMSE mean = 9.68392276763916 RMSE stdev = 0.0
n = 200 Negll mean = 1.0328667163848877 Negll stdev = 0.0
Trial = 0
n = 300 RMSE mean = 8.546952247619629 RMSE stdev = 0.0
n = 300 Negll mean = 1.0026687383651733 Negll stdev = 0.0
Trial = 0
n = 400 RMSE mean = 8.40604019165039 RMSE stdev = 0.0
n = 400 Negll mean = 0.9952229261398315 Negll stdev = 0.0
Trial = 0
n = 500 RMSE mean = 7.727145195007324 RMSE stdev = 0.0
n = 500 Negll mean = 0.9876633286476135 Negll stdev = 0.0
Time taken = 0.020779196156395806 hours


In [8]:
respath = 'Results' #Path to folder for saving results (change as necessary)
if ss_flag:
    respath += '_SS'

mean_filename = respath+'/RMSE_mean_DPKL.npy'
std_filename = respath+'/RMSE_std_DPKL.npy'
np.save(mean_filename,rmse_mean)
np.save(std_filename,rmse_std)

mean_filename = respath+'/Negll_mean_DPKL.npy'
std_filename = respath+'/Negll_std_DPKL.npy'
np.save(mean_filename,negll_mean)
np.save(std_filename,negll_std)
       
print ('Results saved')

Results saved
